# Measured sky check

> Check that the L1 spectra have a measured sky brightness fainter than the observational requirement.

In [ ]:
# |default_exp diagnostics.measured_sky_check

In [ ]:
# |hide

# This prevents warnings from appearing in cell outputs, since it has proved difficult
# to suppress warnings arising from dask workers.
%env PYTHONWARNINGS=ignore

env: PYTHONWARNINGS=ignore


In [ ]:
# |export

import numpy as np
import xarray as xr

from qagmire.data import (
    get_lr_l1_stack_files,
    read_fibre_table_nspec,
    read_l1_data,
    read_primary_header,
)
from qagmire.quality_assurance import Diagnostics
from qagmire.utilities import get_mags_from_spectra, parse_obstemp

To write checks of the data, we create a subclass of `Diagnostics` and implement the `tests` method.

In [ ]:
# |export


class MeasuredSkyCheck(Diagnostics):
    """Measured sky check.

    A reproduction of the SkyTrueCheck class in the weaveio
    [sky_true_limit](https://github.com/bamford/QAG/blob/master/diagnostics/sky_true_limit.py).

    This tests for the following cases:

    * Does the measured sky brightness in the raw spectra satisfy the observational requirement?
    * Does the measured sky brightness vary substantially between the sky fibres in an exposure?
    """

    def __init__(
        self,
        variation_limit: float = 0.2,  # the maximum permissible sky-brightness variation in an exposure
        **kwargs,  # additional keyword arguments are passed to the `Diagnostics` constructor
    ):
        self.ap_area = np.pi * (0.65 * 0.65)
        self.variation_limit = variation_limit
        super().__init__(**kwargs)

    def tests(self, **kwargs):
        files = get_lr_l1_stack_files(**kwargs)
        data = read_l1_data(files)
        # merge in info from fibre_table
        fibre_table = read_fibre_table_nspec(files)
        data = xr.merge((data, fibre_table))
        data = data.where(data["TARGUSE"] == b"S")
        self.fibre_table = fibre_table
        # merge in observational requirements
        hdr = read_primary_header(files)
        obs = parse_obstemp(hdr["OBSTEMP"])
        data = xr.merge((data, obs))
        # limit the dataset to the blue camera
        data = data.sel(filename=(data["CAMERA"] == "BLUE"), drop=True)
        data = data.drop_dims("LAMBDA_R")
        # perform the tests by OBID, rather than filename
        data = data.swap_dims(filename="OBID")

        data["flam"] = data["BLUE_FLUX_NOSS"] * data["BLUE_SENSFUNC"] / self.ap_area
        data["mag_obs"] = get_mags_from_spectra(
            data["LAMBDA_B"], data["flam"], band="GROUND_JOHNSON_V"
        )
        self.data = data

        self.stats = xr.Dataset(
            {
                "median_sky": data["mag_obs"].median("NSPEC"),
                "sky_limit": data["sky_brightness"],
                "sigma_sky": data["mag_obs"].std("NSPEC"),
            }
        )

        tests = [
            {
                "name": "sky_too_bright",
                "description": "Does the measured sky brightness in the raw spectra "
                "satisfy the observational requirement?",
                "test": self.stats["median_sky"] < self.stats["sky_limit"],
            },
            {
                "name": "sky_too_variable",
                "description": "Does the measured sky brightness vary substantially "
                f"(> {self.variation_limit} mag) between the sky fibres for each OB?",
                "test": self.stats["sigma_sky"] > self.variation_limit,
            },
        ]
        return tests

## Demonstration tests

In [ ]:
tests = MeasuredSkyCheck(n_processes=1)
tests.run(date="201*")

Locating and converting where necessary: 100%|██████████| 34/34 [01:01<00:00,  1.81s/it]
Reading netCDF files... took 1.90 s. Size is 15525.568 Mb
Reading files: 100%|██████████| 34/34 [00:03<00:00, 11.31it/s]
Creating Dataset... took 0.24 s. Size is 10.214 Mb
Reading files: 100%|██████████| 34/34 [00:01<00:00, 28.82it/s]
Creating Dataset... took 0.72 s. Size is 0.222 Mb
Tests took 71.84 s to prepare (including reading data).
Tests took 11.87 s to perform.
sky_too_bright:
    Does the measured sky brightness in the raw spectra satisfy the observational requirement?
sky_too_variable:
    Does the measured sky brightness vary substantially (> 0.2 mag) between the sky fibres for each OB?


In [ ]:
tests.summary()

,,,,,,failed,total fails
,,,,,test,sky_too_bright,
OBID,filename,RUN,CAMERA,MJD,NIGHT,,
3170,stack_1002286,1002286,BLUE,57640.9114,20160909,True,1
3175,stack_1002334,1002334,BLUE,57641.1379,20160909,True,1
3191,stack_1002214,1002214,BLUE,57639.8653,20160908,True,1
3217,stack_1002310,1002310,BLUE,57641.0017,20160909,True,1
3295,stack_1002346,1002346,BLUE,57641.1830,20160909,True,1
3346,stack_1002238,1002238,BLUE,57639.9544,20160908,True,1
3372,stack_1002262,1002262,BLUE,57640.1806,20160908,True,1
3380,stack_1002322,1002322,BLUE,57641.0926,20160909,True,1


## Verification

We now do some spot checks to verify and expand upon the above test results. Note that we assigned `self.stats` inside `tests`. This provides a way of accessing the statistics used in the tests, without having to construct them again. However, we still need to recompute the statistics (or whatever is derived from the `self.stats` `DataArray`), which takes a little time.

In [ ]:
tests.stats.to_dataframe()[tests.stats.data_vars]

,median_sky,sky_limit,sigma_sky
OBID,,,
3191,20.603799,21.0,0.074585
3133,20.773965,20.5,0.004181
3346,20.820886,21.0,0.010860
3434,21.248163,21.7,0.005929
3372,21.114255,21.5,0.005882
3170,20.483335,20.5,0.003557
3189,20.549922,20.5,0.019661
3217,20.489997,21.0,0.012291
3380,21.393496,21.7,0.007065


In [ ]:
# |hide
import nbdev

nbdev.nbdev_export()